<a href="https://colab.research.google.com/github/jugernaut/MACTI-programacionparalelo/blob/main/01_OpenMP/05_HilosDeEjecucion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="Teal" face="Comic Sans MS,arial">
  <h1 align="center"><i>Hilos de ejecución</i></h1>
  </font>
  <font color="Black" face="Comic Sans MS,arial">
  <h5 align="center"><i>Profesor: M. en C. Miguel Angel Pérez León</i></h5>
  <h5 align="center"><i>Ayudante: Diana Laura Rojas Cristino</i></h5>
  <h5 align="center"><i>Ayudante: Rodrigo Alejandro Sánchez Morales</i></h5>
  <h5 align="center"><i>Materia: Manejo de Datos</i></h5>
</font>

# Diferencia entre hilo y proceso

Un proceso es aquella tarea (puede estar compuesta de subtareas) que es ejecutada por el microprocesador y en el caso de computadoras con varios procesadores, cada uno de estos CPU's puede estar ejecutando al menos un proceso en un determinado paso de tiempo

Podemos pensar en un proceso como un programa o aplicación en ejecución. Y en este caso una aplicación es un archivo escrito en algún lenguaje de programación que esta siendo ejecutado por el microprocesador.

También puede suceder que en un proceso se puedan ejecutar varias instancias, como cuando se abren varias ventanas de un clock de notas o un navegador. Así, se deduce y es verdad que un proceso contiene un hilo –mínimo el hilo principal que corre en primer plano.

## CPU v.s. GPU

Actualmente los dispositivos de cómputo contienen al menos un *CPU* y dentro de este *CPU* pueden estar contenidos varios núcleos, lo que permite el desarrollo de algoritmos en paralelo.

De igual manera, la mayoría de los dispositivos de cómputo contienen al menos un *GPU* y dentro de este *GPU* pueden existir varios núcleos, la principal diferencia entre ambos (*CPU* y *GPU*) es el propósito para el cuál fueron diseñados

Para fines prácticos (y del curso) podemos pensar que la diferencia principal entre una *CPU* (unidad de procesamiento central) y una *GPU* (**unidad de procesamiento gráfico**) radica en que un *CPU* es un dispositivo de cómputo de propósito general, puede realizar cualquier tipo de cómputo que se le asigne.

Por otro lado un *GPU* esta diseñado para el procesamiento gráfico, lo que significa que la forma en la que procesa información **esta optimizada para trabajar con matrices y vectores**.

<center>
<img src="https://github.com/jugernaut/Numerico2021/blob/desarrollo/Figuras/MonteCarlo/mejor.png?raw=1" width="600">
</center>

<center>
<img src="https://github.com/jugernaut/Numerico2021/blob/desarrollo/Figuras/MonteCarlo/cpu-vs-gpu.jpg?raw=1" width="600">
</center>

# Concurrente v.s. Paralelo

¿Cuál es la diferencia entre concurrencia y
paralelismo?.

Lo primero que hay que decir es: que el paralelismo es una forma de concurrencia; es decir, todo programa paralelo es concurrente.

Una implementación secuencial es la forma tı́pica de solucionar un problema; sin embargo, esta técnica no siempre es la óptima.

El algoritmo secuencial podrá resolver el problema, pero esto no es lo único que nos interesa, sino también en cuánto tiempo lo hace y cuánta memoria necesita. Una desventaja principal es que, no todo algoritmo secuencial puede convertirse a uno paralelo.

En cómputo paralelo en lugar de usar un procesador para solucionar un problema, tendremos $n$ procesadores disponibles resolviendo el mismo problema.

Ası́ el problema se estará resolviendo continuamente, no por intervalos de tiempo, como sucede en la concurrencia, por ello el tiempo invertido en resolver el problema se reduce.

<center>
<img src="https://github.com/jugernaut/ManejoDatos/blob/desarrollo/Imagenes/Simulacion/concurrenteparalelo.png?raw=1" width="800">
</center>

En esta imagen vemos como se distribuye el uso del procesador dependiendo del tipo de algoritmo que se este usando.

<center>
<img src="https://github.com/jugernaut/ManejoDatos/blob/desarrollo/Imagenes/Simulacion/tiempo.png?raw=1" width="800">
</center>

Y finalmente podemos ver la diferencia en tiempo de procesamiento de un algoritmo concurrente y uno en paralelo.

### Conceptos

Ademas de las técnicas previamente mencionadas para poder realizar múltiples tareas dentro de una computadora, existen 2 conceptos relativamente nuevos:

* **Programación en paralelo Multicore**: con la creciente capacidad de cómputo actual existen computadoras que cuentan con más de un núcleo (core) es decir que cuentan con más de un microprocesador. Es en este tipo de equipos en los cuales se pueden ejecutar múltiples procesos (no solo hilos) en paralelo con la finalidad de realizar una tarea en el menor tiempo posible.

* **Programación en paralelo mediante GPU's**: un GPU es una unidad grafica de procesamiento, casi por accidente se descubrió que estas unidades graficas también son útiles para llevar a cabo cómputo en paralelo, incluso han mostrado un mejor desempeño en cálculos de punto flotante que algunos microprocesadores. Sin embargo su desventaja principal es que la memoria no es compartida.

Se sigue de las definiciones anteriores que un proceso se puede ejecutar de manera secuencial o de manera concurrente.

* **Secuencial**: proceso, tarea o algoritmo que tiene que ejecutarse paso a paso, es decir que hasta que no se termine de realizar uno de sus pasos, no se puede iniciar el siguiente.

* **Concurrente**: todo aquel proceso tarea o algoritmo que puede ser dividido en subtareas mas pequeñas que pueden ser realizadas al mismo tiempo. Una vez realizadas las subtareas, los resultados pueden unirse.

# Ley de Amdahl

Sea $f$ la fracción de operaciones en un calculo computacional que sera llevado a cabo de manera secuencial, donde $0\leq f\leq1$. La máxima velocidad $\Psi$ alcanzada mediante programación en paralelo con una computadora con $p$ procesadores enfocados en el mismo calculo es:

$$\Psi\leq\frac{1}{f+(1-f)/p}$$

¿Qué interpretación le das a esta ley?

# *threading*

La biblioteca es un módulo de *Python* que nos permite hacer uso de hilos de manera muy sencilla, basta con importar esta biblioteca y usarla.

La forma más sencilla de usar un `Thread` es crear una instancia con un función de destino y llamar a `start()` para que comience a funcionar.

In [ ]:
import threading

def trabajador():
  # esto es lo único que hace nuestro trabajador
  print('Trabajador')

# se genera una lista de hilos
hilos = []
for i in range(5):
  # agregamos cada hilo a la lista
  t = threading.Thread(target=trabajador)
  hilos.append(t)
  # se ejecuta cada hilo, ¿de forma concurrente o en paralelo?.
  t.start()

Trabajador
Trabajador
Trabajador
Trabajador
Trabajador


Esta forma es la forma más sencilla de hacer uso de hilos, sin embargo se esta realizando de manera funcional, es decir que no estamos haciendo uso de la POO.

Otra forma de hacer uso de los hilos, es mediante POO en la cual vamos a crear una clase que herede de la clase `threading` y que sea capas de funcionar mediante hilos.

Volvamos a ver el ejemplo del carrito de compras, a continuación se muestra la versión secuencial de la misma.

In [ ]:
import time

# clase que ayuda a contener el nombre y articulos que compra un cliente
class Cliente(object):

  def __init__(self, nombre):
    # nombre del cliente
    self.nombre = nombre
    # articulos que compro el cliente
    self.carrito = []

# clase que simula el cobro que llevaria a cabo una empresa de compras en linea
class Amazon(object):

  # VARIABLE DE CLASE O ESTATICA
  # diccionario con el catalogo de articulos y su precio
  catalogo = {'PS5':10000, 'Audifonos':2500, 'Halo':1500, 'HDD':900,
              'Laptop':25000, 'GTX1000':6000, 'SmartWatch':7500}

  def __init__(self, nombre):
    self.nombre = nombre

  # metodo que simula el cobro de un cliente
  def cobrar(self, cliente):
    print('Comenzamos a cobrar al cliente :', cliente.nombre)
    # se revisa el carrito del cliente y el catalogo
    total = 0
    # cada articulo se cobra
    for articulo in cliente.carrito:
      print('Se cobra :',articulo, self.catalogo[articulo])
      # se hace una pausa de 2 segundos para simular el cobro
      time.sleep(2)
      # total almacena el total de los articulos del cliente
      total += self.catalogo[articulo]
    print('Total del cliente', cliente.nombre,':', total)


if __name__ == "__main__":
  inicio = time.time()
  cliente1 = Cliente('Mike')
  cliente1.carrito = ['PS5', 'SmartWatch', 'Audifonos']

  amazon = Amazon('cobrador 1')
  amazon.cobrar(cliente1)

  cliente2 = Cliente('Ivan')
  cliente2.carrito = ['GTX1000', 'HDD', 'GTX1000', 'Laptop']

  amazon.cobrar(cliente2)

  cliente3 = Cliente('Jonathan')
  cliente3.carrito = ['Laptop', 'PS5', 'Audifonos', 'Halo', 'HDD',
                      'GTX1000', 'SmartWatch']

  amazon.cobrar(cliente3)
  fin = time.time()

  print("Este algoritmo tomo: ", (fin-inicio)*1000)


Comenzamos a cobrar al cliente : Mike
Se cobra : PS5 10000
Se cobra : SmartWatch 7500
Se cobra : Audifonos 2500
Total del cliente Mike : 20000
Comenzamos a cobrar al cliente : Ivan
Se cobra : GTX1000 6000
Se cobra : HDD 900
Se cobra : GTX1000 6000
Se cobra : Laptop 25000
Total del cliente Ivan : 37900
Comenzamos a cobrar al cliente : Jonathan
Se cobra : Laptop 25000
Se cobra : PS5 10000
Se cobra : Audifonos 2500
Se cobra : Halo 1500
Se cobra : HDD 900
Se cobra : GTX1000 6000
Se cobra : SmartWatch 7500
Total del cliente Jonathan : 53400
Este algoritmo tomo:  28038.564205169678


Y en la siguiente celda se muestra la versión concurrente (en paralelo si el h.w. lo permite).

In [ ]:
import threading
import time

# clase que ayuda a contener el nombre y articulos que compra un cliente
class Cliente(object):

  def __init__(self, nombre):
    # nombre del cliente
    self.nombre = nombre
    # articulos que compro el cliente
    self.carrito = []

class Amazon(threading.Thread):

  # VARIABLE DE CLASE O ESTATICA
  # diccionario con el catalogo de articulos y su precio
  catalogo = {'PS5':10000, 'Audifonos':2500, 'Halo':1500, 'HDD':900,
              'Laptop':25000, 'GTX1000':6000, 'SmartWatch':7500}

  def __init__(self, nombre, cliente):
    super(Amazon, self).__init__()
    self.nombre = nombre
    self.cliente = cliente

  # metodo que simula el cobro de un cliente
  def run(self):
    print('Comenzamos a cobrar al cliente :', self.cliente.nombre)
    # se revisa el carrito del cliente y el catalogo
    total = 0
    # cada articulo se cobra
    for articulo in self.cliente.carrito:
      time.sleep(2)
      print('Se cobra :',articulo, self.catalogo[articulo])
      # total almacena el total de los articulos del cliente
      total += self.catalogo[articulo]
    print('Total del cliente', self.cliente.nombre,':', total)

if __name__ == "__main__":
  inicio = time.time()
  cliente1 = Cliente('Mike')
  cliente1.carrito = ['PS5', 'SmartWatch', 'Audifonos']

  cliente2 = Cliente('Ivan')
  cliente2.carrito = ['GTX1000', 'HDD', 'GTX1000', 'Laptop']

  cliente3 = Cliente('Jonathan')
  cliente3.carrito = ['Laptop', 'PS5', 'Audifonos', 'Halo', 'HDD',
                      'GTX1000', 'SmartWatch']

  amazon1 = Amazon("Trabajador 1", cliente1)
  # se ejecuta cada hilo de manera concurrente
  amazon1.start()

  amazon2 = Amazon("Trabajador 2", cliente2)
  # se ejecuta cada hilo de manera concurrente
  amazon2.start()

  amazon3 = Amazon("Trabajador 3", cliente3)
  # se ejecuta cada hilo de manera concurrente
  amazon3.start()

  # esperamos a que todos los hilos hayan terminado
  amazon1.join()
  amazon2.join()
  amazon3.join()

  fin = time.time()
  print("Este algoritmo tomo: ", (fin-inicio)*1000)



Comenzamos a cobrar al cliente : MikeComenzamos a cobrar al cliente :
 Ivan
Comenzamos a cobrar al cliente : Jonathan
Se cobra : PS5 Se cobra : GTX1000 6000
10000
Se cobra : Laptop 25000
Se cobra : Se cobra : SmartWatch 7500
HDD 900
Se cobra : PS5 10000
Se cobra :Se cobra : Se cobra : Audifonos 2500
 Audifonos 2500
Total del cliente Mike : 20000
GTX1000 6000
Se cobra : Halo 1500
Se cobra : Laptop 25000
Total del cliente Ivan : 37900
Se cobra : HDD 900
Se cobra : GTX1000 6000
Se cobra : SmartWatch 7500
Total del cliente Jonathan : 53400
Este algoritmo tomo:  14022.159814834595


Vale la pena notar la diferencia en la ejecución de ambos algoritmos y sobretodo la diferencia en el tiempo que tardan ambas versiones.

# Referencias

* Michaell J. Quuin: Parallel Programming in C with OpenMP and MPI.
* GPU Parallel Program Development Using CUDA: Tolga Soyata
* Referencias Libro Web: Introduccion a Python.
* Referencias Dongarra Foster: Source Book of parallel computing.
* [Diferencias CPU Físico v.s. Lógico](https://www.daniloaz.com/es/diferencias-entre-cpu-fisica-cpu-logica-core-nucleo-thread-hilo-y-socket/)